In [37]:
import pandas as pd
import os
from imageLabeler import ImageLabeler

def run_labeller():
    # lists files in your data folder
    files = os.listdir('./data')

    # Filter out non-image files
    image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # only run when are ready to label everything, hard to debug, takes about 25-40 min to do this depending on your speed
    for image_file in image_files:
        image_path = os.path.join('./data', image_file)
        print(f"Labeling image: {image_file}")

        # label the image
        ImageLabeler(image_path)

run = False
if run:
    run_labeller()

In [ ]:
df = pd.read_csv('labeled_data.csv')

df = pd.DataFrame([df.columns.values] + df.values.tolist())
df.columns = ['file_name', 'character', 'position']

df['file_name'] = df['file_name'].str.replace('./data/', '')
order_values = list(range(1, 12)) * (len(df) // 11) + list(range(1, (len(df) % 11) + 1))

df['order'] = order_values

df['position'] = df['position'].apply(lambda x: tuple(map(float, x.strip('()').split(', '))))

df['top_left'] = df['position'].apply(lambda x: (x[0], x[1]))
df['bottom_right'] = df['position'].apply(lambda x: (x[2], x[3]))

df.drop(columns=['position'], inplace=True)

df.to_csv('labeled_data_cleaned.csv', index=False)

df.head()

In [ ]:
data = pd.read_csv('./labeled_data_cleaned.csv')
data.head()

In [ ]:
data

In [41]:
def check_character_alignment(csv_file_path):
    # Load the CSV file
    df = pd.read_csv(csv_file_path)

    # Number of characters to check and length of file extension (e.g., for '.png')
    n_characters = 11
    file_extension_length = 4

    # List to store names of problematic files
    problematic_files = []

    # Process each file in the DataFrame
    for file_name, group in df.groupby('file_name'):
        # Extract the last 11 characters from the file name (excluding the extension)
        last_characters = file_name[-(n_characters + file_extension_length):-file_extension_length]

        # Concatenate characters from the DataFrame in the given order
        concatenated_characters = ''.join(group.sort_values(by='order')['character'].tolist())

        # Check if the concatenated characters match the last 11 characters of the filename
        if concatenated_characters != last_characters:
            problematic_files.append(file_name)

    return problematic_files

# Path to the CSV file
csv_file_path = './labeled_data_cleaned.csv'  # Replace with your actual CSV file path

# Perform the analysis
problematic_files = check_character_alignment(csv_file_path)

# Displaying the problematic file names
print("Problematic Files:", problematic_files)


Problematic Files: []
